In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy.ndimage import label
from tqdm import tqdm
import matplotlib.pyplot as plt

MAX_STEPS = 10

# List of desired ratios of parameters to training samples
target_ratios = [
    0.10,  # Moderate underparameterization
    0.20,  # Balanced regime
    0.50,  # Mild overparameterization
    1.00,  # 1:1 parameter-to-sample ratio
    2.00,  # Clear overparameterization
    5.00,  # Strong overparameterization
]

# Ensure output directories exist
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def generate_percolation_lattice(size, p):
    return np.random.choice([0, 1], (size, size), p=[1-p, p]).astype(np.uint8)

def check_percolation(lattice):
    labeled, _ = label(lattice)
    top = set(labeled[0]) - {0}
    bottom = set(labeled[-1]) - {0}
    left = set(labeled[:,0]) - {0}
    right = set(labeled[:,-1]) - {0}
    top_bottom = float(bool(top & bottom))
    left_right = float(bool(left & right))
    return top_bottom, left_right

def first_coarse_graining(binary_lattice, dim):
    t = torch.tensor(binary_lattice, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    patches = F.unfold(t, kernel_size=dim, stride=dim)
    patches = patches.permute(0, 2, 1)
    coarse_vals = patches.mean(dim=2)
    H, W = binary_lattice.shape
    new_h, new_w = H // dim, W // dim
    # Duplicate for two identical channels
    coarse_vals = coarse_vals.view(1, 1, new_h, new_w).repeat(1, 2, 1, 1)
    return coarse_vals.squeeze(0)

class DirectionalPercolationModel(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        self.rule = nn.Sequential(
            nn.Linear(2 * dim * dim, 32),  # 2*b^2 inputs
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 2),  # Two outputs
            nn.Sigmoid()
        )

    def forward(self, x, max_steps=MAX_STEPS):
        b, c, H, W = x.shape
        for _ in range(max_steps):
            if H < self.dim or W < self.dim:
                break
            patches = F.unfold(x, kernel_size=self.dim, stride=self.dim)
            patches = patches.permute(0, 2, 1).contiguous()
            # Process 2*b^2 inputs per patch
            out = self.rule(patches.view(-1, 2 * self.dim * self.dim))
            new_h, new_w = H // self.dim, W // self.dim
            # Reshape to two channels
            x = out.view(b, 2, new_h, new_w)
            _, _, H, W = x.shape
        
        # Final adaptive pooling to 1x1 to get two values
        x = F.adaptive_avg_pool2d(x, (1, 1))
        return x.view(b, 2)

def prepare_dataset(N, sizes):
    data = []
    half = int(N/2)
    for _ in tqdm(range(half), desc="Generating uniform data"):
        p = np.random.uniform(0, 1)
        size = np.random.choice(sizes)
        L = generate_percolation_lattice(size, p)
        tb, lr = check_percolation(L)
        data.append((L, (tb, lr)))
    for _ in tqdm(range(half), desc="Generating critical data"):
        p = np.random.uniform(0.55, 0.65)
        size = np.random.choice(sizes)
        L = generate_percolation_lattice(size, p)
        tb, lr = check_percolation(L)
        data.append((L, (tb, lr)))
    return data

def train_epoch(model, device, data, batch_size, opt, crit):
    model.train()
    total_loss = 0.0
    for i in tqdm(range(0, len(data), batch_size), desc="Training", leave=False):
        batch = data[i:i+batch_size]
        groups = {}
        for x, y in batch:
            cg = first_coarse_graining(x, DIM)
            size = tuple(cg.shape[-2:])
            groups.setdefault(size, []).append((cg, y))
        for group in groups.values():
            lattices, labels = zip(*group)
            inputs = torch.stack(lattices).to(device)
            # Convert labels to tensor [batch, 2]
            targets = torch.tensor(labels, dtype=torch.float32, device=device)
            opt.zero_grad()
            outputs = model(inputs)
            loss = crit(outputs, targets)
            loss.backward()
            opt.step()
            total_loss += loss.item() * len(group)
    return total_loss / len(data)

def compute_loss(model, device, data, crit):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for x, y in tqdm(data, desc="Evaluating", leave=False):
            cg = first_coarse_graining(x, DIM)
            inp = cg.unsqueeze(0).to(device)
            tgt = torch.tensor([y], dtype=torch.float32, device=device)
            out = model(inp)
            loss = crit(out, tgt)
            total_loss += loss.item()
    return total_loss / len(data)

def run_experiment(base, epochs, param_ratio):
    global DIM
    DIM = base
    SIZES = [base**2, base**3, base**4]

    # Setup directory structure
    out_dir = f"results/base_{base}"
    ensure_dir(out_dir)

    # Initialize model and compute parameter count
    model = DirectionalPercolationModel(DIM).to(DEVICE)
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    train_samples = int(num_params / param_ratio)
    test_samples = train_samples  # Set test samples equal to training samples
    
    print(f"Base={base}, Ratio={param_ratio}: params={num_params}, samples={train_samples}")

    train_data = prepare_dataset(train_samples, SIZES)
    test_data = prepare_dataset(test_samples, SIZES)

    opt = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    crit = nn.BCELoss()
    train_losses, test_losses = [], []

    for epoch in range(1, epochs+1):
        tr_loss = train_epoch(model, DEVICE, train_data, batch_size=100, opt=opt, crit=crit)
        te_loss = compute_loss(model, DEVICE, test_data, crit)
        train_losses.append(tr_loss)
        test_losses.append(te_loss)
        print(f"Epoch {epoch}/{epochs} - Train Loss: {tr_loss:.4f}, Test Loss: {te_loss:.4f}")

    # Plot and save
    plt.figure(figsize=(10, 6))
    x = range(1, epochs+1)
    plt.plot(x, train_losses, 'b-o', label='Training Loss')
    plt.plot(x, test_losses, 'g--s', label='Test Loss')
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('BCE Loss', fontsize=14)
    plt.title(f'Directional Percolation Loss (base {base}, ratio {param_ratio})', fontsize=16)
    plt.legend()
    plt.tight_layout()
    
    filepath = os.path.join(out_dir, f"loss_ratio{str(param_ratio).replace('.', '_')}.png")
    plt.savefig(filepath)
    plt.close()

if __name__ == "__main__":
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bases = [3, 4, 5]
    epochs = 500
    
    # Create base directories first
    for base in bases:
        ensure_dir(f"results/base_{base}")
    
    # Run experiments
    for ratio in target_ratios:
        for base in bases:
            run_experiment(base, epochs, ratio)

Base=3, Ratio=0.1: params=2786, samples=27860


Generating critical data: 100%|█████████████████████████████████████████████████████████████| 13930/13930 [00:03<00:00, 4396.33it/s]
                                                                                                                                    

Epoch 1/500 - Train Loss: 0.4507, Test Loss: 0.3183


Epoch 2/500 - Train Loss: 0.3703, Test Loss: 0.3138


Epoch 3/500 - Train Loss: 0.3489, Test Loss: 0.3024


...

Epoch 496/500 - Train Loss: 0.1816, Test Loss: 0.1702


Epoch 497/500 - Train Loss: 0.2313, Test Loss: 0.3477


Epoch 498/500 - Train Loss: 0.1956, Test Loss: 0.1749


Epoch 499/500 - Train Loss: 0.3264, Test Loss: 0.2616


Epoch 500/500 - Train Loss: 0.1896, Test Loss: 0.1737
